In [1]:
# !pip install jiwer pyctcdecode "pypi-kenlm" --upgrade -qq

In [2]:
# for cpu inference
# !pip uninstall onnxruntime-gpu -y
# !pip install onnxruntime -qq

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import onnxruntime as rt
from sk import predict,load_model,load_decoder,labels
from pathlib import Path
import jiwer
import pandas as pd
import string
from fastcore.basics import partialler
from IPython.display import Audio
import multiprocessing
pd.set_option('max_colwidth', 400)

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
path = "/content/MyDrive/neem/onnx/conformer_small.onnx"
# path = "/content/model"

In [4]:
model = load_model(path)

In [5]:
from pyctcdecode import build_ctcdecoder
import kenlm
kenlm_model = kenlm.Model("/content/out.trie.klm")
decoder = build_ctcdecoder(
    labels,
    kenlm_model, 
    alpha=0.5,
    beta=1.0, 
    ctc_token_idx=labels.index("_")
)

In [7]:
fn = "/content/test-bahasa/"
preds,files,entropy,timesteps = predict(model,fn,decoder)

100%|██████████| 780/780 [02:27<00:00,  5.29it/s]


In [8]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    char_label = [char for char in label]
    char_i = [char for char in i]
    data.append([j,label,i,jiwer.compute_measures(label,i)['wer'],jiwer.compute_measures(char_label,char_i)['wer']])

In [9]:
df = pd.DataFrame(data)
df.columns = ["path","label","pred","wer","cer"]
df = df.sort_values("cer",ascending=False)
df.head(50)

,path,label,pred,wer,cer
276,/content/test-bahasa/wattpad-audio-wattpad-646.wav,who are you,ayu,1.000000,0.666667
428,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu power ordonterceti powers,0.750000,0.478261
338,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ118-000005.wav,boleh fuckers quora sila tutup fuck tentang iq,boleh pakar sekolah sila tutup pak tentang aku,0.500000,0.358974
243,/content/test-bahasa/wattpad-audio-wattpad-638.wav,itu andra batin libra,itu anda makin liberal,0.750000,0.277778
163,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-alau.wav,sebut perkataan alau,sibut keadaan alau,0.666667,0.277778
563,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-ampe.wav,sebut perkataan ampe,sebuk pakatan ampa,1.000000,0.277778
53,/content/test-bahasa/-home-husein-speech-bahasa-tolong-sebut-coalition.wav,tolong sebut coalition,tolong sebut kolifen,0.333333,0.250000
383,/content/test-bahasa/wattpad-audio-wattpad-88.wav,udah ga cape kita caw yuk,udagacapi kita chayok,0.833333,0.250000
493,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-aksesori.wav,sebut perkataan aksesori,sebat perkataan accessory,0.666667,0.227273
46,/content/test-bahasa/iium-audio-iium-45.wav,memang salah aku,memang selaku,0.666667,0.214286


In [11]:
df["wer"].mean(),df["cer"].mean()

(0.13840744719524614, 0.03795880139918297)

# Grid search

In [ ]:
# TODO